<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Juan Aranda
- Nombre de alumno 2: Nicolás Crocco

### **Link de repositorio de GitHub:** `https://github.com/Juanaranda/Aranda_Crocco`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 108.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    #path = '/content/drive/MyDrive/Laboratorios/Lab_07'
    path = '/content/drive/MyDrive/12vo Semestre/Laboratorio de Programación Científica/Laboratorios/Lab_07'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle(path+"/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
# En uso
def custom_features(dataframe_in):
    # Calculamos Length (L)
    grouped = dataframe_in.groupby('Customer ID')['InvoiceDate'].agg(['min', 'max'])
    grouped['Length'] = (grouped['max'] - grouped['min']).dt.days

    # Calculamos Recency (R)
    current_date = dataframe_in['InvoiceDate'].max()
    grouped['Recency'] = (current_date - grouped['max']).dt.days + 1

    # Calculamos Frequency (F)
    grouped['Frequency'] = dataframe_in.groupby('Customer ID').size()

    # Calculamos Monetary (M)
    dataframe_in['Monetary'] = dataframe_in['Price'] * dataframe_in['Quantity']
    grouped['Monetary'] = np.round(dataframe_in.groupby('Customer ID')['Monetary'].mean(), decimals=2)

    # Calculamos Periodicity (P)
    dataframe_in['IVT'] = dataframe_in.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    grouped['Periodicity'] = np.round(dataframe_in.groupby('Customer ID')['IVT'].std(), decimals=2)

    # Eliminamos columnas y filas que no sirven
    dataframe_in.drop(columns='IVT', inplace=True)
    grouped.drop(columns=['min', 'max'], inplace=True)
    grouped = grouped[grouped['Length'] > 0]

    # Rellenamos los valores NaN con ceros si es necesario
    grouped = grouped.fillna(0)

    return pd.DataFrame(grouped)

In [5]:
# Obtenemos el resultado
df_retail_lrmfp = custom_features(df_retail)
df_retail_lrmfp.head(10)

,Length,Recency,Frequency,Monetary,Periodicity
Customer ID,,,,,
12346.0,196,165,33,11.30,21.72
12347.0,37,3,71,18.64,4.42
12349.0,181,43,102,26.19,16.20
12352.0,16,11,18,19.10,3.88
12356.0,44,16,83,42.90,3.66
12358.0,356,11,58,46.88,32.96
12359.0,308,61,117,21.91,15.16
12360.0,276,15,88,18.08,19.63
12361.0,220,98,19,16.91,35.48


**Resúesta de Pauta:**

La función comienza tomando un DataFrame de entrada y agrupa los datos por el id del cliente. Luego, calcula la diferencia de tiempo entre la primera y la última compra de cada cliente en términos de días, lo que representa el "Length (L)" de la relación con la empresa.<br>
A continuación, se obtiene la fecha más reciente en el que cada cliente realizó una compra, y se calcula la "Recency (R)" para cada uno de ellos.<br>
Luego, la función determina la "Frecuency (F)" de compra de cada cliente contando el número de transacciones en el conjunto de datos para cada cliente.<br>
La función sigue por calcular el valor monetario promedio de las compras de cada cliente ("Monetary (M)"). Multiplica el precio y la cantidad de productos comprados en cada transacción para obtener el valor monetario de esa transacción y luego calcula el promedio de estos valores para cada cliente.<br>
Después se calcula la "Periodicity (P)" de compra para cada cliente. Esto se hace calculando la diferencia en días entre las fechas de compra sucesivas y luego calculando la desviación estándar de estas diferencias para cada cliente.
Finalmente, la función elimina las columnas temporales y de cálculo que ya no son necesarias, como la columna 'IVT', y filas donde la longitud de la relación con la empresa es igual a cero.<br>
Finalmente, devuelve un nuevo dataframe con las métricas solicitadas para cada cliente.

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [6]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Calcula el valor mínimo y máximo para cada columna
        min_values = X.min()
        max_values = X.max()

        # Aplica la transformación Min-Max a cada columna
        X_transformed = (X - min_values) / (max_values - min_values)

        return X_transformed

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [7]:
# Definir una función callable que obtiene las características LRMFP
def select_LRMFP_features(dataframe):
    return dataframe[['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']]

# Paso 1: Obtén las características LRMFP y escala con MinMax
step1 = FunctionTransformer(func=select_LRMFP_features)

# Paso 2: Aplicar MinMax
step2 = MinMax()

# Paso 3: Aplica T-SNE para reducción de dimensionalidad
step3 = TSNE(n_components=2, random_state=42)

# Crea el pipeline
pipeline = Pipeline([
    ('step1', step1),
    ('step2', step2),
    ('step3', step3)
])

# Ajusta el pipeline a los datos
X_reduced = pipeline.fit_transform(df_retail_lrmfp)

# Crea un DataFrame para las componentes reducidas
reduced_df = pd.DataFrame(data=X_reduced, columns=['Component 1', 'Component 2'])

# Gráfica las componentes obtenidas en la reducción de dimensionalidad con Plotly
fig = px.scatter(reduced_df, x='Component 1', y='Component 2', title="T-SNE Reduced Dimensionality")
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [8]:
# Lista para almacenar las sumas de los cuadrados de las distancias
sse = []

# Rango de valores para el número de clusters
k_values = list(range(1, 21))

for k in k_values:
    # Creamos el pipeline
    pipeline = Pipeline([
        ('features', step1),
        ('scaler', step2),
        ('kmeans', KMeans(n_clusters=k))
    ])

    # Aplicamos el pipeline
    pipeline.fit(df_retail_lrmfp)

    # Obtenemos la suma de los cuadrados de las distancias
    sse.append(pipeline.named_steps['kmeans'].inertia_)

# Creamos el plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=k_values, y=sse, mode='lines+markers'))
fig.update_layout(
    title='Método del codo',
    xaxis_title='Número de clusters (k)',
    yaxis_title='SSE'
)
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

En base al método del codo que se puede ver en el gráfico anterior, decidimos utilizar 4 clusters. Esto debido a que la varianza en los datos disminuye considerablemente a partir del 4to cluster.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [18]:
# Crear el pipeline con los pasos definidos
pipeline = Pipeline([
      ('features', step1),
      ('scaler', step2),
      ('kmeans', KMeans(n_clusters=4))
])
# Ajustamos el pipeline
pipeline.fit(df_retail_lrmfp)

# Obtenemos la labels
cluster_labels = pipeline.named_steps['kmeans'].labels_

# Creamos una nueva columna 'Cluster' en el DataFrame de LRMFP
df_retail_lrmfp['Cluster'] = cluster_labels

# Calcular los promedios por cluster para cada atributo
cluster_means = df_retail_lrmfp.groupby('Cluster').mean()

# Mostrar la tabla de promedios por cluster
print(np.round(cluster_means, 2))


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



         Length  Recency  Frequency  Monetary  Periodicity
Cluster                                                   
0        329.45    20.94     222.00     37.51        20.10
1         91.01   206.38      56.29     32.65        13.84
2        207.96    55.87     100.76     27.64        21.62
3         63.38    37.02      71.17     25.94         8.34


Analizando la tabla anterior, podemos decir que sí se crearon 4 clusters de clientes coherentes. Las cluster creados se pueden definir de la siguiente manera:
- Cluster 0: Posee un lenght y frequency muy altos, lo que indica que son clientes que compran hace mucho tiempo en la tienda de Mr. Lepin, y además, con harta frecuencia, lo que se condice también con el alto nivel de periodicity. Son los clientes que además tiene una recency más baja, lo que indicaría que compraron hace menos tiempo y, por otro lado, son el grupo de clientes que gasta más dinero en la tienda. Podríamos decir que este el cluster de *clientes frecuentes*.
- Cluster 1: Este cluster difiere mucho del anterior en varios aspectos. En primer lugar, posee una length y frecuency considerablemente más baja, lo que indeicaría que son clientes relativamente nuevos y que no compran muy seguido en la tienda, esto último se condice también con un score de periodicity más bajo que en el cluster anterior. Por otro lado, tienen un nivel de recency muy alto, el más alto de los 4 clusters, lo que quiiere decir que realizaron su última compra hace más tiempo. En comparación con los otros clusters, poseen un nivel monetario relativamente alto, por lo que este no nos indicaría mucho. Podríamos decir que este es el cluster de *clientes inactivos*, considerando sobretodo los niveles de lenght, recency y frecuency.
- Cluster 3: Los clientes de este cluster, al igual que los del primero, son clientes más antiguos, ya que poseen un nivel de lemnght bastanto alto en comparación a los otros clusters. Tienen un recency, frecuency y monetary más bien moderados, y la periodicity más alta de todas, esto nos indica que este grupo de clientes realiza compras de forma regular. Podríamos definir este cluster como los *clientes regulares*, valga la redundancia.
- Cluster 4: En este cluster nos encontramos con clientes más nuevos, que realizaron su primera compra hace un menor tiempo que los clientes regulares, y, además, tienen una frecuencia un tanto mayor que ese mismo segmento de clientes y un gasto monetrio relativamente parecido. Por otro laod, poseen la preiiodicity más baja de todas, estando 5 puntos por debajo que la del grupo de clientes inactivo. Esto nos indicaría que son clientes esporádicos, podriamos definirlo como el grupo de *clientes casuales*.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [21]:
# Nombres de los clusters
cluster_labels_mapping = {
    0: 'Clientes Frecuentes',
    1: 'Clientes Inactivos',
    2: 'Clientes Regulares',
    3: 'Clientes Casuales'
}

# Asignamos etiquetas a los clusters
df_retail_lrmfp['Cluster_Label'] = df_retail_lrmfp['Cluster'].map(cluster_labels_mapping)

# Creamos el plot
fig = px.scatter(x=X_reduced[:, 0], y=X_reduced[:, 1], color=df_retail_lrmfp['Cluster_Label'])
fig.update_layout(
    title='Clusters según LRMFP',
    xaxis_title='Componente 1',
    yaxis_title='Componente 2'
)
fig.show()

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>